## Import library

In [1]:
%run functions

c:\Users\GIGA\AppData\Local\Programs\Python\Python310\lib\site-packages\outdated\utils.py:14: OutdatedPackageWarning: The package outdated is out of date. Your version is 0.2.1, the latest is 0.2.2.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


## Load data

In [30]:
loader = DataLoader()
loader.GetData('final')
d = loader.GetShuffled(1000)
d.head()

,用户ID,学号,姓名,班级,院系,任课教师,Class,Prompt,score,text
3249,1003247070,21120892,谭浩然,2021-22冬季通用英语C2周健华1024期末,机电工程与自动化学院,1002658409(周健华),C,19121440,9.0,"As we know,reading has a lot of advantages f..."
1221,1003246436,21120258,陈辰,2021-22冬季通用英语B2朱虹1028期末,社会学院,1002658416(朱虹),B,19121440,11.9,"Nowadays, with the growing development o..."
1600,1003247064,21120886,杨浩辰,2021-22冬季通用英语B2薛清1013期末,通信与信息工程学院,1002658414(薛清),B,19121413,10.8,It has been more than 7 months when coming int...
2845,1003250658,21124485,聂子晨,2021-22冬季通用英语C2张季红1014期末,音乐学院,1002658404(张季红),C,19121436,9.4,How to learn a foreign language，different ...
3422,1003249020,21122847,陶钧,2021-22冬季通用英语C2张季红1002期末,计算机工程与科学学院,1002658404(张季红),C,1000561535,9.6,"Today,we are amaziing at the frequency of ..."


In [31]:
d = d.loc[d['score'] > 0,:]

## Regression
### Split for train and test index

In [32]:
ind = list(d.index)
trainIn,testIn = ind[:2838],ind[2838:]

### Define regression function

In [5]:
def regression(x,plt=False):
    TrainX,TestX = [x[i] for i in trainIn],[x[i] for i in testIn]
    TrainY,TestY = d['score'][trainIn],d['score'][testIn]
    model = xgb.XGBRegressor(learning_rate=0.042)
    model.fit(TrainX,TrainY)
    PredY = model.predict(TestX)
    PredTrainY = model.predict(TrainX)
    #print('test RMSE: {:.3f}, test EV: {:.3f}%'.format(mean_squared_error(TestY,PredY,squared=False),explained_variance_score(TestY,PredY)*100))
    #print('train RMSE: {:.3f}, train EV: {:.3f}%'.format(mean_squared_error(TrainY,PredTrainY,squared=False),explained_variance_score(TrainY,PredTrainY)*100))
    if plt:
        g = sns.scatterplot(x=TestY,y=PredY)
        g.set_ylabel('predicted score')
        g.set_xlabel('true score')
        g.set_ylim((0,14))
        g.set_xlim((0,14))
        g = sns.lineplot(x=[0,14],y=[0,14],color='black')
        g.lines[0].set_linestyle("--")
    return (mean_squared_error(TestY,PredY,squared=False),explained_variance_score(TestY,PredY)*100,mean_squared_error(TrainY,PredTrainY,squared=False),explained_variance_score(TrainY,PredTrainY)*100)

### Prepare data
#### Embeddings

In [6]:
xe = torch.load('features/final-meanp.pt')

In [7]:
xe = [np.array(i).reshape(1024) for i in xe]

### Fine-grained 

In [8]:
xf = []
with open('features/final-winter-fine.csv') as f:
    raw = f.readlines()[1:]

In [9]:
for i in range(len(raw)):
    l = raw[i]
    l = l.rstrip('\n').split(',')[2:]
    l = [float(i) for i in l]
    if i in list(d.index):
        xf.append(np.array(l,dtype=float).reshape(143))
    else:
        xf.append([0])

#### Large-grained

In [10]:
with open('features/final-winter-l2sca.txt') as f:
    l2 = f.readlines()[1:]
large = dict()
for l in l2:
    i = int(l.split(',')[0].split('.')[0])
    feat = l.rstrip('\n').split(',')[-14:]
    feat = [float(s) for s in feat]
    large[i] = feat
xl = []
for i in range(3830):
    xl.append(large[i])

#### Sim

In [11]:
f = np.load('features/final_winter_sent.npz')
sim_list = [f[a] for a in f.files]
sim_raw_list = []
for n in f.files:
    values = []
    m = f[n]
    for row in range(len(m)):
        for col in range(row+1,len(m)):
            values.append(m[row,col])
    sim_raw_list.append(np.array(values))
sim_mean = []
sim_max = []
sim_min = []
for matrix in sim_raw_list:
    if len(matrix) > 1:
        sim_mean.append(np.average(matrix))
        sim_max.append(np.max(matrix))
        sim_min.append(np.min(matrix))
    else:
        sim_mean.append(0)
        sim_max.append(0)
        sim_min.append(0)
cluster_num = []
cluster_avg = []
for k in range(len(sim_list)):
    sim = sim_list[k]
    if len(sim) > 1:
        sim_thres = sim.copy()
        for i in range(len(sim_thres)):
            for j in range(len(sim_thres)):
                if sim_thres[i,j] <= np.percentile(np.array(sim_raw_list[k]),50):
                    sim_thres[i,j] = 0
        res = mcl.run_mcl(sim_thres,inflation=2)
        clusters = mcl.get_clusters(res)
        cluster_num.append(len(clusters))
        cluster_avg.append(len(sim)/len(clusters))
    else:
        cluster_num.append(0)
        cluster_avg.append(0)

In [12]:
xs = []
for i in range(3830):
    a = [sim_max[i],sim_mean[i],sim_min[i],cluster_avg[i],cluster_num[i]]
    xs.append(a)

#### Embedding + sim

In [13]:
xes = []
for i in range(3830):
    a = np.concatenate((xe[i],xs[i]))
    xes.append(a)

#### Fine-grained + sim

In [14]:
xfs = []
for i in range(3830):
    a = np.concatenate((xf[i],xs[i]))
    xfs.append(a)

#### Large-grained + sim

In [15]:
xls = []
for i in range(3830):
    a = np.concatenate((xl[i],xs[i]))
    xls.append(a)

### Embedding + fine

In [16]:
xef = []
for i in range(3830):
    a = np.concatenate((xe[i],xf[i]))
    xef.append(a)

#### Embedding + large

In [17]:
xel = []
for i in range(3830):
    a = np.concatenate((xe[i],xl[i]))
    xel.append(a)

#### Embedding + fine + sim

In [18]:
xefs = []
for i in range(3830):
    a = np.concatenate((xef[i],xs[i]))
    xefs.append(a)

#### Embedding + large + sim

In [19]:
xels = []
for i in range(3830):
    a = np.concatenate((xel[i],xs[i]))
    xels.append(a)

### Get performances

In [20]:
names = ['xe','xl','xf','xs','xls','xfs','xef','xel','xefs','xels']
trainRMSE, trainEV, testRMSE, testEV = [],[],[],[]
x = [xe,xl,xf,xs,xls,xfs,xef,xel,xefs,xels]
for i in range(len(x)):
    res = regression(x[i])
    trainRMSE.append(res[2])
    trainEV.append(res[3])
    testRMSE.append(res[0])
    testEV.append(res[1])
r = pd.DataFrame({'test RMSE':testRMSE,
                  'test EV':testEV,
                  'train RMSE':trainRMSE,
                  'trainEV':trainEV},index=names)

In [21]:
r

,test RMSE,test EV,train RMSE,trainEV
xe,0.595541,80.765008,0.258681,97.456532
xl,1.171061,22.621189,0.788199,67.808214
xf,0.738300,70.200623,0.486430,88.370021
xs,1.100947,32.746863,0.882350,59.440234
xls,0.915106,53.603523,0.643854,78.874673
xfs,0.747658,69.391599,0.479792,88.715167
xef,0.469843,88.407490,0.243815,97.847575
xel,0.590192,81.149761,0.253959,97.581720
xefs,0.472231,88.291384,0.242782,97.875622
xels,0.587450,81.318499,0.251243,97.656052


In [33]:
names = ['xl','xf','xs','xls','xfs','xe','xefs']
trainRMSE, trainEV, testRMSE, testEV = [],[],[],[]
x = [xl,xf,xs,xls,xfs,xe,xefs]
for i in range(len(x)):
    res = regression(x[i])
    trainRMSE.append(res[2])
    trainEV.append(res[3])
    testRMSE.append(res[0])
    testEV.append(res[1])
r = pd.DataFrame({'test RMSE':testRMSE,
                  'test EV':testEV,
                  'train RMSE':trainRMSE,
                  'trainEV':trainEV},index=names)

In [34]:
r

,test RMSE,test EV,train RMSE,trainEV
xl,1.156707,31.824992,0.787036,66.745279
xf,0.750017,71.609637,0.487064,87.916827
xs,1.110637,36.683227,0.877543,58.438118
xls,0.967426,52.233496,0.641521,78.279841
xfs,0.745341,72.005501,0.483758,88.093282
xe,0.601101,81.981004,0.259017,97.356732
xefs,0.490159,88.229262,0.241503,97.833462


In [44]:
for n in names:
    print('{:.3f}'.format(r['train RMSE'][n]))

0.787
0.487
0.878
0.642
0.484
0.259
0.242
